In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 50.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=b987887cd81c72fbce868eb210a8a687eb1ded378fe2cf7a54b56991fb7300b7
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
## Ignore warning

In [4]:
from warnings import filterwarnings
filterwarnings('ignore')

In [5]:
# import libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
appName= "hive_pyspark"
master= "local"

In [6]:
spark = SparkSession.builder \
	.master(master).appName(appName).enableHiveSupport().getOrCreate()

In [7]:
## Question 1

In [8]:
data=spark.read.csv("boston.csv",header=True)
data.show(10)
data.write.saveAsTable("boston")

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|  18|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|         24|
|0.027310001|   0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729|   0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001|   0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999|   0|2.

In [9]:
# Question 2

In [10]:
df=spark.sql("select * from boston")
df.show()

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|  18|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|         24|
|0.027310001|   0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729|   0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001|   0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999|   0|2.

In [11]:
# Question 3

In [13]:
for col in df.dtypes:
    print(col[0]+" , "+col[1])

CRIM , string
ZN , string
INDUS , string
CHAS , string
NOX , string
RM , string
AGE , string
DIS , string
RAD , string
TAX , string
PT , string
B , string
LSTAT , string
MV , string


In [14]:
from pyspark.sql.functions import col
df = df.select([col(column).cast('double') for column in df.columns])

In [15]:
for col in df.dtypes:
    print(col[0]+" , "+col[1])

CRIM , double
ZN , double
INDUS , double
CHAS , double
NOX , double
RM , double
AGE , double
DIS , double
RAD , double
TAX , double
PT , double
B , double
LSTAT , double
MV , double


In [16]:
vec = "cf"
assembler = VectorAssembler(inputCols=df.columns, outputCol=vec)
df_vec = assembler.transform(df).select(vec)

In [17]:
matrix = Correlation.corr(df_vec, vec)

In [18]:
matrix.collect()[0]["pearson({})".format(vec)].values

array([ 1.        , -0.20046922,  0.40658343, -0.05589158,  0.42097173,
       -0.2192467 ,  0.35273425, -0.37967009,  0.62550515,  0.58276431,
        0.28994564, -0.38506395,  0.45562148, -0.38830461, -0.20046922,
        1.        , -0.53382819, -0.04269672, -0.51660371,  0.31199059,
       -0.56953734,  0.66440822, -0.31194783, -0.31456332, -0.39167853,
        0.17552031, -0.41299458,  0.36044534,  0.40658343, -0.53382819,
        1.        ,  0.06293803,  0.76365146, -0.39167586,  0.64477851,
       -0.70802699,  0.5951293 ,  0.7207602 ,  0.38324764, -0.35697654,
        0.60379972, -0.48372517, -0.05589158, -0.04269672,  0.06293803,
        1.        ,  0.0912028 ,  0.09125123,  0.08651777, -0.09917578,
       -0.00736824, -0.03558652, -0.12151517,  0.04878849, -0.0539293 ,
        0.17526018,  0.42097173, -0.51660371,  0.76365146,  0.0912028 ,
        1.        , -0.30218816,  0.73147011, -0.76923012,  0.61144056,
        0.66802321,  0.18893272, -0.38005064,  0.59087892, -0.42

In [19]:
# Question 4

In [20]:
vectorAssembler = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(df)
vhouse_df = vhouse_df.select(['features', 'MV'])
vhouse_df.show(3)

+--------------------+-----------+
|            features|         MV|
+--------------------+-----------+
|[0.00632,18.0,2.3...|       24.0|
|[0.027310001,0.0,...|21.60000038|
|[0.02729,0.0,7.07...|34.70000076|
+--------------------+-----------+
only showing top 3 rows



In [21]:
splits = vhouse_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [22]:
lr = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-0.048887329157300374,0.013568368027519023,-0.03692333637204433,1.1287989938379854,-4.35050047585973,4.929688582509553,0.0,-0.5014573794010677,0.0,0.0,-0.7071721579271399,0.010575795143024591,-0.41361996781650373]
Intercept: 10.744106321066608


In [23]:
# Question 5

In [24]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 4.821918
r2: 0.720490


In [26]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","MV","features").show(5)
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="MV",metricName="r2")
print("R2 on test = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
| 30.24187802839799|       24.0|[0.00632,18.0,2.3...|
| 38.72243625700208|       50.0|[0.01381,80.0,0.4...|
|30.281929031173902|29.10000038|[0.01439,60.0,2.9...|
| 28.02147409317832|       24.5|[0.01501,80.0,2.0...|
|31.761920850284916|32.90000153|[0.01778,95.0,1.4...|
+------------------+-----------+--------------------+
only showing top 5 rows

R2 on test = 0.67806


In [27]:
test_result = lr_model.evaluate(test_df)
print("RMSE on test = %g" % test_result.rootMeanSquaredError)

RMSE on test = 5.2852


In [28]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

numIterations: 10
objectiveHistory: [0.49999999999999956, 0.42968990498784077, 0.23034998081952984, 0.2087219612015162, 0.17882951926142848, 0.17611146610643663, 0.17565894983955913, 0.17530181802341435, 0.17525166046150956, 0.17517175911206084, 0.1750666017370606]
+--------------------+
|           residuals|
+--------------------+
|  0.6998875130860682|
| -5.8120413540149585|
|  0.3512659643844316|
|   4.319961371615278|
|  0.5602154968559141|
| 0.06719563569807363|
|   9.330700480451434|
|   8.763980036692516|
|  2.7250905514284263|
|  -3.479921811968211|
|   5.809199425019337|
|   10.29469696491305|
| -1.8212636156465898|
|-0.05522772956271993|
| -10.319407970921965|
|   3.429701147118301|
|  1.7516669001829683|
|  -1.081049908233112|
| -0.3266110978117531|
|  2.5833616680264733|
+--------------------+
only showing top 20 rows



In [29]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","MV","features").show()

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
| 30.24187802839799|       24.0|[0.00632,18.0,2.3...|
| 38.72243625700208|       50.0|[0.01381,80.0,0.4...|
|30.281929031173902|29.10000038|[0.01439,60.0,2.9...|
| 28.02147409317832|       24.5|[0.01501,80.0,2.0...|
|31.761920850284916|32.90000153|[0.01778,95.0,1.4...|
|21.636852517324968|20.10000038|[0.019649999,80.0...|
| 36.85248267772286|42.29999924|[0.02177,82.5,2.0...|
|28.160213798943282|23.89999962|[0.025429999,55.0...|
|25.433462331641557|21.60000038|[0.027310001,0.0,...|
| 29.63829347977137|31.20000076|[0.03049,55.0,3.7...|
|20.714433214079538|       17.5|[0.031129999,0.0,...|
| 24.13061800160149|19.39999962|[0.03466,35.0,6.0...|
|30.356296671413318|       28.5|[0.035020001,80.0...|
|25.389178805295188|24.79999924|[0.036589999,25.0...|
| 23.21926321390626|20.70000076|[0.037379999,0.0,...|
|24.487581832519773|20.60000